In [4]:
#!/usr/bin/env python3
import os
import pandas as pd

# 1. Verzeichnis mit den CSV-Dateien
csv_dir = 'output/csv'

# 2. Pfade zu den einzelnen Dateien
mem_file    = os.path.join(csv_dir, 'nist_mem.csv')
kem_file    = os.path.join(csv_dir, 'nist_time_kem.csv')
sig_file    = os.path.join(csv_dir, 'nist_time_sig.csv')
output_file = os.path.join(csv_dir, 'nist_all.csv')

# 3. Einlesen der CSV-Dateien
mem_df = pd.read_csv(mem_file)
kem_df = pd.read_csv(kem_file)
sig_df = pd.read_csv(sig_file)

# 4. Spaltennamen vereinheitlichen
kem_df = kem_df.rename(columns={
    'keypair (μs)': 'keygen_time',
    'enc (μs)':     'enc_time',
    'dec (μs)':     'dec_time'
})
sig_df = sig_df.rename(columns={
    'keypair (μs)': 'keygen_time',
    'sign (μs)':    'enc_time',
    'verify (μs)':  'dec_time'
})

# 5. Typ-Spalte hinzufügen
kem_df['Typ'] = 'kem'
sig_df['Typ'] = 'sig'

# 6. Zusammenführen der Zeitdaten
time_df = pd.concat([kem_df, sig_df], ignore_index=True)

# 7. Runden-Spalte als Integer formatieren
time_df['Runden'] = time_df['Runden'].astype(float).astype(int)

# 8. Spaltenauswahl inkl. Typ (Typ an dritter Stelle)
time_df = time_df[['Model', 'Algorithmus', 'Typ', 'Runden', 'keygen_time', 'enc_time', 'dec_time']]

# 9. Merge mit den Memory-Daten
final_df = pd.merge(mem_df, time_df, on=['Model', 'Algorithmus'], how='left')

# 10. Zeilen mit fehlenden Zeitwerten entfernen (prüfen auf keygen_time)
final_df = final_df.dropna(subset=['keygen_time'])

# 11. Spaltenreihenfolge sicherstellen (Typ und Runden als 3. und 4. Spalte)
cols = ['Model', 'Algorithmus', 'Typ', 'Runden'] + [c for c in final_df.columns if c not in ('Model', 'Algorithmus', 'Typ', 'Runden')]
final_df = final_df[cols]

# 12. Speichern der kombinierten CSV (Runden als ganze Zahl schreiben)
if not os.path.isdir(csv_dir):
    os.makedirs(csv_dir, exist_ok=True)
final_df.to_csv(output_file, index=False)
print(f"Ergebnis in {output_file} gespeichert.")


Ergebnis in output/csv\nist_all.csv gespeichert.
